In [ ]:
# PyTorch tutorial

# PyTorch is a machine learning library that gives us two gifts
# 1) Tensors (high-dim matrices) powered by GPU
# 2) Neural networks powered by autodifferentiation (backprop done for you)

# Documentation: https://pytorch.org/docs/stable/index.html

#References:
# Deeplizard's Neural Network Programming 
#the detailed tutorial can be found here https://deeplizard.com/learn/playlist/PLZbbT5o_s2xrfNyHZsM6ufI0iZENK9xgG
# python-engineer's PyTorch tutorial: https://github.com/python-engineer/pytorchTutorial
# Srijith Rajamohan's word2vec implementation in PyTorch (CBOW, SG): https://srijithr.gitlab.io/post/word2vec/

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import urllib.request
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import word_tokenize
import sklearn
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances

# Pytorch Data Structures

**Tensor**: A generalization of scalars, vectors, matrices, ...

**Rank**: number of dimensions present within the tensor.

**Axis**: a specific dimension of a tensor.


"Length Of An Axis tells us how many indexes are available along each axis"

## Understanding tensors

In [ ]:
x = torch.rand(1) # scalar 1x1 #different from the scalar of rank zero
print(x)

x = torch.rand(5) # vector 1x5
print(x)

x = torch.rand(2,4) # matrix 2x4
print(x)

x = torch.rand(2,2,2,2) #tensor 2x2x2x2
print(x)

# all zeros
x = torch.zeros(2,2)
# negatives okay
x = torch.randn(2,2)
# ints only!
x = torch.randint(2,3,size=(2,3))
x = torch.ones(2,2)

tensor([0.4949])
tensor([0.9607, 0.8121, 0.0107, 0.0293, 0.0391])
tensor([[0.3530, 0.6742, 0.4603, 0.2955],
        [0.5086, 0.9686, 0.3841, 0.2667]])
tensor([[[[0.1812, 0.8004],
          [0.6736, 0.6264]],

         [[0.5456, 0.7868],
          [0.1551, 0.5023]]],


        [[[0.3425, 0.2995],
          [0.4833, 0.3866]],

         [[0.2595, 0.9813],
          [0.6798, 0.2350]]]])


In [ ]:
#shapes matter
x = torch.rand(1).reshape(1,1)
x

tensor([[0.4853]])

In [ ]:
# Operations
y = torch.rand(2, 2)
x = torch.rand(2, 2)

# elementwise addition
z = x + y
# torch.add(x,y)

# substraction
z = x - y
z = torch.sub(x, y)

# multiplication
z = x * y
z = torch.mul(x,y) #elementwise #must be broadcastable

#matrix multiplication
tensor1 = torch.randn(3, 4)
tensor2 = torch.randn(4, 3)
torch.matmul(tensor1, tensor2)

# division
z = x / y
z = torch.div(x,y)

# Slicing
x = torch.rand(5,3)
print(x)
print(x[:, 0]) # all rows, column 0
print(x[1, :]) # row 1, all columns
print(x[1,1]) # element at 1, 1

tensor([[0.9098, 0.7598, 0.2127],
        [0.7989, 0.0864, 0.7626],
        [0.0841, 0.0679, 0.2153],
        [0.7047, 0.3256, 0.9144],
        [0.6795, 0.1113, 0.7348]])
tensor([0.9098, 0.7989, 0.0841, 0.7047, 0.6795])
tensor([0.7989, 0.0864, 0.7626])
tensor(0.0864)


In [ ]:
#when doing elementwise operations and the two variable tensors do not have the same shape, pytorch will use broadcasting
#find more about broadcasting here: https://deeplizard.com/learn/video/QscEWm0QTRY

In [ ]:


# Extract an element
print(x[1,1].item())

# NumPy conversion
x = torch.rand(2,2)
y = x.numpy()
print(type(y))

z1 = torch.from_numpy(y) #sharing the memory space with the numpy ndarray
z2 = torch.tensor(y) #a copy
print(type(z1))
print(type(z2))

0.08643519878387451
<class 'numpy.ndarray'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>


## Pytorch attributes and functions for tensors

In [ ]:
print(x.shape)
print(x.device)

torch.Size([2, 2])
cpu


### Shape changes
**Squeezing** removes the dimensions or axes that have a length of one.

In [ ]:
 x = torch.zeros(2, 1, 2, 1, 2)
 print(x.shape)
 y = torch.squeeze(x)
 print(y.shape)
 torch.squeeze(x, dim=1).shape #can specify which dimension to squeeze

torch.Size([2, 1, 2, 1, 2])
torch.Size([2, 2, 2])


torch.Size([2, 2, 1, 2])

### Autograd

In [ ]:
# autograd

# requires grad equals true lets us compute gradients on the tenor
x = torch.tensor([2,3,5], dtype=float, requires_grad=True)
y =(5 * x**2).sum()



# When we finish our computation we can call .backward() and have all the gradients computed automatically
# The gradient for this tensor will be accumulated into .grad attribute
y.backward()
#print(z.grad) # dz/dz
print(x.grad) # dz/dx

tensor([20., 30., 50.], dtype=torch.float64)


caveats
* autograd requires computational resources and can take time.

* disable autograd for model eval by writing your evaluation code in 
```
with torch.no_grad():
```



In [ ]:
w=torch.tensor(np.arange(10),requires_grad=True, dtype=float).reshape(2,5)
print(w.requires_grad)
with torch.no_grad():
  print('-'*60)
  print((w**2).requires_grad)
  print(w.requires_grad)
  print('-'*60)
print(w.requires_grad)

True
------------------------------------------------------------
False
True
------------------------------------------------------------
True


# Neural Nets in Pytorch

In [ ]:
# example of a single forward pass and backpropagation

#input: x is 1
#output: y is 2
x = torch.tensor(1.0)
y = torch.tensor(2.0)

w = torch.tensor(1.0, requires_grad=True)

# forward pass to compute loss
y_predicted = w * x
loss = (y_predicted - y)**2
print(loss)

# backward pass to compute gradient dLoss/dw
loss.backward() #gradient computation dLoss/dwhatever
print(w.grad) #gradient of w, dLoss/dw

# update weights
# next forward and backward pass...

# continue optimizing:
# update weights, this operation should not be part of the computational graph
with torch.no_grad():
    #learning rate times the gradient (a little change in w wrt to the Loss, to make it smaller)
    w -= 0.01 * w.grad 
# don't forget to zero the gradients
w.grad.zero_()
print(w)

tensor(1., grad_fn=<PowBackward0>)
tensor(-2.)
tensor(1.0200, requires_grad=True)


In [ ]:
# lets look at an actual model where we do stuff manually

# Linear regression
# f = w * x 
# here : f = 2 * x
X = np.array([1, 2, 3, 4], dtype=np.float32)
Y = np.array([2, 4, 6, 8], dtype=np.float32)
w = 0

# model output
def forward(x):
    return w * x

# loss = MSE
def loss(y, y_pred):
    return ((y_pred - y)**2).mean()
  
# J = MSE = 1/N * (w*x - y)**2
# dJ/dw = 1/N * 2x(w*x - y)
def gradient(x, y, y_pred):
    return np.dot(2*x, y_pred - y).mean()

print(f'Prediction before training: f(5) = {forward(5):.3f}')

# Training
learning_rate = 0.01
n_iters = 10


Prediction before training: f(5) = 0.000


In [ ]:
for epoch in range(n_iters):
    # predict = forward pass
    y_pred = forward(X)

    # loss
    l = loss(Y, y_pred)
    
    # calculate gradients
    dw = gradient(X, Y, y_pred)

    # update weights
    w -= learning_rate * dw

    if epoch % 2 == 0:
        print(f'epoch {epoch+1}: w = {w:.3f}, loss = {l:.8f}')
     
print(f'Prediction after training: f(5) = {forward(5):.3f}')

epoch 1: w = 1.200, loss = 30.00000000
epoch 3: w = 1.872, loss = 0.76800019
epoch 5: w = 1.980, loss = 0.01966083
epoch 7: w = 1.997, loss = 0.00050331
epoch 9: w = 1.999, loss = 0.00001288
Prediction after training: f(5) = 9.999


In [ ]:
# lets replace the manually computed gradient with autograd
X = torch.tensor([1,2,3,4], dtype=torch.float32)
Y = torch.tensor([2,4,6,8], dtype=torch.float32)
w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True)

for epoch in range(n_iters):
    # predict = forward pass
    y_pred = forward(X)

    # loss
    l = loss(Y, y_pred)

    # calculate gradients = backward pass
    l.backward() #dl/deverything

    #shouldn't be part of the gradient checking!!! this is a separate updating rule
    with torch.no_grad():
        w -= learning_rate * w.grad #dl/dw
    
    # zero the gradients after updating
    w.grad.zero_()

    if epoch % 2 == 0:
        print(f'epoch {epoch+1}: w = {w.item():.3f}, loss = {l.item():.8f}')
print(f'Prediction after training: f(5) = {forward(5):.3f}')

epoch 1: w = 0.300, loss = 30.00000000
epoch 3: w = 0.772, loss = 15.66018772
epoch 5: w = 1.113, loss = 8.17471695
epoch 7: w = 1.359, loss = 4.26725292
epoch 9: w = 1.537, loss = 2.22753215
Prediction after training: f(5) = 8.031


In [ ]:
X = torch.tensor([1,2,3,4], dtype=torch.float32)
Y = torch.tensor([2,4,6,8], dtype=torch.float32)
w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True)

#only one layer, this is a little trivial
n_samples, n_features = X.shape
input_size = n_features
output_size = n_features

model = nn.Linear(input_size, output_size)

'''
class LinearRegression(nn.Module):
  def __init__(self, in_dim, out_dim):
    super(LinearRegression, self).__init__()
    self.lin = nn.Linear(in_dim, out_dim)
  def forward(self, x):
    return self.lin(x)
model = LinearRegression(input_size, output_size)
'''

# mean squared error (implicitly knows (truth, predicted)) - a callable function
loss = nn.MSELoss()

#stochastic gradient descent, optimize the parameters in the list with the learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# 3) Training loop
for epoch in range(n_iters):
    # predict = forward pass
    y_predicted = model(X)
    
    # loss
    l = loss(Y, y_predicted)

    # calculate gradients = backward pass
    l.backward()

    # update weights
    optimizer.step()

    # zero the gradients after updating
    optimizer.zero_grad()

    if epoch % 2 == 0:
        print('epoch ', epoch+1, ': w = ', w, ' loss = ', l)

print(f'Prediction after training: f(5) = {model(X_test).item():.3f}')

epoch  1 : w =  tensor(1.6063, requires_grad=True)  loss =  tensor(33.4131, grad_fn=<MseLossBackward>)
epoch  3 : w =  tensor(1.6063, requires_grad=True)  loss =  tensor(16.0947, grad_fn=<MseLossBackward>)
epoch  5 : w =  tensor(1.6063, requires_grad=True)  loss =  tensor(7.7563, grad_fn=<MseLossBackward>)
epoch  7 : w =  tensor(1.6063, requires_grad=True)  loss =  tensor(3.7415, grad_fn=<MseLossBackward>)
epoch  9 : w =  tensor(1.6063, requires_grad=True)  loss =  tensor(1.8085, grad_fn=<MseLossBackward>)
Prediction after training: f(5) = 8.174


In [ ]:
torch.manual_seed(1)

CONTEXT_SIZE = 3
EMBEDDING_DIM = 10


test_sentence = """Empathy for the poor may not come easily to people who never experienced it. They may blame the victims and insist their predicament can be overcome through determination and hard work.
But they may not realize that extreme poverty can be psychologically and physically incapacitating — a perpetual cycle of bad diets, health care and education exacerbated by the shaming and self-fulfilling prophecies that define it in the public imagination.
Gordon Parks — perhaps more than any artist — saw poverty as “the most savage of all human afflictions” and realized the power of empathy to help us understand it. It was neither an abstract problem nor political symbol, but something he endured growing up destitute in rural Kansas and having spent years documenting poverty throughout the world, including the United States.
That sensitivity informed “Freedom’s Fearful Foe: Poverty,” his celebrated photo essay published in Life magazine in June 1961. He took readers into the lives of a Brazilian boy, Flavio da Silva, and his family, who lived in the ramshackle Catacumba favela in the hills outside Rio de Janeiro. These stark photographs are the subject of a new book, “Gordon Parks: The Flavio Story” (Steidl/The Gordon Parks Foundation), which accompanies a traveling exhibition co-organized by the Ryerson Image Centre in Toronto, where it opens this week, and the J. Paul Getty Museum. Edited with texts by the exhibition’s co-curators, Paul Roth and Amanda Maddox, the book also includes a recent interview with Mr. da Silva and essays by Beatriz Jaguaribe, Maria Alice Rezende de Carvalho and Sérgio Burgi.
""".split()

ngrams = []
for i in range(len(test_sentence) - CONTEXT_SIZE):
    tup = [test_sentence[j] for j in np.arange(i + 1 , i + CONTEXT_SIZE + 1) ]
    ngrams.append((test_sentence[i],tup))
# print the first 3, just so you can see what they look like
print(ngrams)

vocab = set(test_sentence)
print("Length of vocabulary",len(vocab))
word_to_ix = {word: i for i, word in enumerate(vocab)}

class SkipgramModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(SkipgramModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.linear2 = nn.Linear(128, context_size * vocab_size)
        #self.parameters['context_size'] = context_size

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))  # -1 implies size inferred for that index from the size of the data
        #print(np.mean(np.mean(self.linear2.weight.data.numpy())))
        out1 = F.relu(self.linear1(embeds)) # output of first layer
        out2 = self.linear2(out1)           # output of second layer
        #print(embeds)
        log_probs = F.log_softmax(out2, dim=1).view(CONTEXT_SIZE,-1)
        return log_probs
      
    def predict(self,input):
        context_idxs = torch.tensor([word_to_ix[input]], dtype=torch.long)
        res = self.forward(context_idxs)
        res_arg = torch.argmax(res)
        res_val, res_ind = res.sort(descending=True)
        indices = [res_ind[i][0] for i in np.arange(0,3)]
        for arg in indices:
            print( [ (key, val) for key,val in word_to_ix.items() if val == arg ])

loss_function = nn.NLLLoss()
model = SkipgramModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(550):
    total_loss = 0
    #------- Embedding layers are trained as well here ----#
    #lookup_tensor = torch.tensor([word_to_ix["poor"]], dtype=torch.long)
    #hello_embed = model.embeddings(lookup_tensor)
    #print(hello_embed)
    # -----------------------------------------------------#

    #model.predict('psychologically')

    for context, target in ngrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        print(context,target)


        context_idxs = torch.tensor([word_to_ix[context]], dtype=torch.long)
        print("Context id",context_idxs)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)
        #print(log_probs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        target_list = torch.tensor([word_to_ix[w] for w in target], dtype=torch.long)
        loss = loss_function(log_probs, target_list)
        #print(loss)

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    print(total_loss)

[('Empathy', ['for', 'the', 'poor']), ('for', ['the', 'poor', 'may']), ('the', ['poor', 'may', 'not']), ('poor', ['may', 'not', 'come']), ('may', ['not', 'come', 'easily']), ('not', ['come', 'easily', 'to']), ('come', ['easily', 'to', 'people']), ('easily', ['to', 'people', 'who']), ('to', ['people', 'who', 'never']), ('people', ['who', 'never', 'experienced']), ('who', ['never', 'experienced', 'it.']), ('never', ['experienced', 'it.', 'They']), ('experienced', ['it.', 'They', 'may']), ('it.', ['They', 'may', 'blame']), ('They', ['may', 'blame', 'the']), ('may', ['blame', 'the', 'victims']), ('blame', ['the', 'victims', 'and']), ('the', ['victims', 'and', 'insist']), ('victims', ['and', 'insist', 'their']), ('and', ['insist', 'their', 'predicament']), ('insist', ['their', 'predicament', 'can']), ('their', ['predicament', 'can', 'be']), ('predicament', ['can', 'be', 'overcome']), ('can', ['be', 'overcome', 'through']), ('be', ['overcome', 'through', 'determination']), ('overcome', ['thr

AssertionError: ignored